In [ ]:
path="/home/drmohammad/Documents/LLM/dataset/my-llm"

In [ ]:
# huggingface-cli login --token

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
df=pd.read_csv("experiment/data.csv")
train, test = train_test_split(df, test_size=0.1)
train.to_csv("experiment/train.csv",index=False)
test.to_csv("experiment/test.csv",index=False)

In [3]:
!autotrain llm --train --project_name my-llm --model meta-llama/Llama-2-7b-hf --data_path "/home/drmohammad/Documents/LLM/llm testing/experiment" --train_split "train" --valid_split "test" --text_column "text" --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 10 --num_train_epochs 3 --trainer sft

bin /home/drmohammad/anaconda3/envs/hugging1/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
2023-07-24 19:32:35.541 | INFO     | autotrain.cli.run_llm:run:446 - Running LLM
2023-07-24 19:32:35.541 | INFO     | autotrain.cli.run_llm:run:447 - Train: True
/home/drmohammad/Documents/LLM/llm testing/experiment <class 'str'>
Loading train set
2023-07-24 19:32:35.541 | INFO     | autotrain.trainers.clm:train:37 - loading dataset from csv
Loading test set
2023-07-24 19:32:38.228 | INFO     | autotrain.trainers.clm:train:51 - loading dataset from csv
Using pad_token, but it is not set yet.
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████████████| 2/2 [00:01<00:00,  1.12it/s]
/home/drmohammad/anaconda3/envs/hugging1/lib/python3.9/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version

In [ ]:
!autotrain llm --train --project_name my-llm --model meta-llama/Llama-2-7b-hf --data_path "/home/drmohammad/Documents/LLM/llm testing/experiment" --train_split "train" --valid_split "test" --text_column "text" --use_peft --use_int4 --learning_rate 2e-4 --train_batch_size 10 --num_train_epochs 1 --trainer sft

In [1]:
!autotrain llm --train --project_name my-llm --model meta-llama/Llama-2-7b-hf --data_path "/home/drmohammad/Documents/LLM/llm testing/experiment" --train_split "train" --valid_split "test" --text_column "text" --use_peft --use_int4 --learning_rate 2e-4 --auto_find_batch_size True --num_train_epochs 1 --trainer sft

bin /home/drmohammad/anaconda3/envs/hugging1/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
usage: autotrain <command> [<args>]
AutoTrain advanced CLI: error: unrecognized arguments: True


In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = r"/home/drmohammad/Documents/LLM/llm testing/experiment/my-llm"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

sequences = pipeline(
    'Bitcoin wasn\'t built to live on exchanges. what is the response of this text?\n',
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=200,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")


In [ ]:
from datasets import load_dataset
import pandas as pd
df = pd.read_csv("bitcoin-sentiment-tweets.csv")

In [ ]:
data = load_dataset("json", data_files=r"/home/drmohammad/Documents/LLM/llm testing/alpaca-bitcoin-sentiment-dataset.json")
data["train"]

Create data prompt for the model

In [ ]:
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
### Instruction:
{data_point["instruction"]}
### Input:
{data_point["input"]}
### Response:
{data_point["output"]}"""

def data_gen(ex):
    ex['text']=generate_prompt(ex)
    return ex

In [ ]:
train_data = (
    data["train"].map(data_gen)
)

In [ ]:
df=pd.DataFrame(train_data)
df.to_csv("data.csv",index=False)

Inference

In [ ]:
import torch
model_path = r"/home/drmohammad/Documents/LLM/llm testing/experiment/my-llm"

In [ ]:
from transformers import AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = LlamaTokenizer.from_pretrained(model_path)
 
model = LlamaForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=True,
    device_map="auto",
)



model.eval()
inputs = tokenizer("Buddy if I had whatever series of 19th diseases Bannon clearly has I'd want to be a bitcoin too. what is the response of this text?", return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])



Inference

In [17]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
import torch
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
model_path = r"/home/drmohammad/Documents/LLM/llm testing/experiment/my-llm"
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=True,
    device_map="auto",
)
pipe = pipeline(
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)
local_llm = HuggingFacePipeline(pipeline=pipe)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/drmohammad/anaconda3/envs/hugging1/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/drmohammad/anaconda3/envs/hugging1/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/drmohammad/anaconda3/envs/hugging1/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/home/drmohammad/anaconda3/envs/hugging1/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/drmohammad/anaconda3/envs/hugging1/lib/libcudart.so.11.0'), PosixPath('/home/drmohammad/anaconda3/envs/hugging1/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.67s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerat

: 

In [ ]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
summary_template = """
         given the Linkedin information {information} about a person from I want you to create:
         1. a short summary
         2. two interesting facts about them
     """

summary_prompt_template = PromptTemplate(
        input_variables=["information"], template=summary_template
    )
chain = LLMChain(llm=local_llm, prompt=summary_prompt_template)
print(chain.run(information="what is the book?"))